In [18]:
# Imports
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import pandas_datareader as pdr
import pandas as pd
import requests
import csv
from bs4 import BeautifulSoup as BS4


from bokeh.plotting import figure, output_file, show
from math import pi
from bokeh.layouts import gridplot, column, row
from bokeh.io import show
from bokeh.models import CustomJS, TextInput, Toggle, PreText, RadioButtonGroup

# Settings
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 1000)

In [19]:
def read_input():
    ticker = input(print("Stock Symbol to search: ",end = ''))
    print(ticker)
    return ticker

In [20]:
def get_data(ticker,start,end):
    if start==0:
        try:
            ohlcv = ticker.history(period="max")
        except:
            return None
    else:
        try:
            ohlcv = yf.download(ticker, start=start, end=end)
        except:
            return None
    return ohlcv

In [21]:
# EMA Signals Calculation
# Values used: EMA-buy, EMA-sell
# Zones - denote buy and sell zones for transactions
# Positions - denote exact buy and sell positions
def generate_EMA_signals(ohlcv, fast, slow):

    # Returns the DataFrame of symbols containing the signals to go long, sell or hold (1, -1 or 0)       
    signals = pd.DataFrame(index=ohlcv.index)

    # Create the set of short and long buy and sell exponential moving averages over the respective periods
    signals["Fast_EMA"]=ohlcv["Open"].ewm(span=fast,min_periods=fast, adjust = False).mean()
    signals["Slow_EMA"]=ohlcv["Open"].ewm(span=slow,min_periods=slow, adjust = False).mean()

    # Finding signals for EMA buy and sell [no filters - whenever any signal is found, it shows]- act as sell zones
    signals['zones'] = np.where(signals["Fast_EMA"]/signals['Slow_EMA'] >= 1, 1.0, -1.0)

    # Contains specific positions where trade is to be made
    conditions_bns = [(signals['zones'] == 1) & (signals['zones'].shift(1) != 1), (signals['zones'] == -1) & (signals['zones'].shift(1) == 1)]
    choices_bns = [1,-1]
    signals['temp'] = np.select(conditions_bns, choices_bns, default=0.0)
    signals['positions'] = 0.0 
    signals['positions'][1:] = signals['temp'][:-1]

    return signals

In [22]:
# RSI Signals Calculation [no cooldown required]
# Values used: rsi lower limit, rsi higher limit
# Zones - denote buy and sell zones for transactions later on
# Positions - denote exact buy and sell positions
def generate_RSI_signals(ohlcv, rsi_low, rsi_high):

    # Returns the DataFrame of symbols containing the signals to go long, sell or hold (1, -1 or 0)       
    signals = pd.DataFrame(index=ohlcv.index)
    signals['signal_buy'] = 0.0
    signals['signal_sell'] = 0.0

    # Find RSI value
    delta = ohlcv['Open'].diff()
    up = delta.clip(lower=0)
    down = -1*delta.clip(upper=0)
    ema_up = up.ewm(com=13, adjust=False).mean()
    ema_down = down.ewm(com=13, adjust=False).mean()
    signals['RSI'] = 100-(100/(1+(ema_up/ema_down)))
    
    # Add reference value next to RSI value
    signals["ref"]=signals['RSI'].shift(3)
    
    # Calculate ratio of RSI/RSI_Ref
    signals['ratio']=signals['RSI']/signals['ref']

    # Add value for RSI signals - not filtered, all signals shown
    conditions  = [ (signals['RSI'] > 30) & (signals['ref'] < 30), (signals['RSI'] < 80) & (signals['ref'] > 80) ]
    choices     = [ 1, -1]
    signals['positions'] = np.select(conditions, choices, default=0.0)

    # Add Zones for evaluation
    signals['zone']=signals['positions'].replace(to_replace=0, method='ffill')

    # Specify positions for buy/sell
    conditions_bns = [(signals['zone'] == 1) & (signals['zone'].shift(1) != 1), (signals['zone'] == -1) & (signals['zone'].shift(1) == 1)]
    choices_bns = [1,-1]
    signals['positions'] = np.select(conditions_bns, choices_bns, default=0.0)
    
    return signals

In [23]:
def bnh(ohlcv):
    ret = (10**6/ohlcv["Open"][0])*ohlcv["Open"][-1]-10**6
    return ret

In [24]:
def calculate_pnl(ohlcv,signals):
    capital = 10**6
    amt = 0
    length = len(ohlcv)
    for i in range(length):

        #Sell
        if signals.positions[i] == -1:
            capital=amt*ohlcv["Open"][i]
        #Buy
        if signals.positions[i] == 1:
            amt=capital/ohlcv["Open"][i]
            capital = 0

    if capital == 0:
        capital = amt*ohlcv["Open"][-1]
    returns = capital - 10**6
    return returns

In [25]:
# Generates Historical highs and lows in the chart
# Highest and lowest values in time
def generate_hilo(ohlcv):
    df = ohlcv.copy()
    hi = df.High.max()
    df['hi'] = np.where(df.High == hi, 1, 0)
    lo = df.Low.min()
    df['lo'] = np.where(df.Low == lo, -1, 0)
    df['positions'] = df.hi + df.lo

    return df

In [26]:
# Display function to display all elements of the plots
# EMA lines, RSI plot, Vol plot, buy-sell points, hammers, shooting stars, historical high-lows
def disp(ohlcv,signals,rsi, hilo,y):

    df = ohlcv.copy()

    # Result region
    string = y[0]
    string = "RESULTS: "+ string + "\t\tFast_EMA: " + str(y[1]) + "\t\tSlow_EMA: "+ str(y[2])+ " \nReturn on Strategy: " + str(y[3]) + "\t\tReturn on BuynHold: " + str(y[4]) + "\t\tFeasibility of Strategy: " + str(y[5])
    result = PreText(text=string,width=150, height=100)

    # Candlestick
    inc = df.Close > df.Open
    dec = df.Open > df.Close
    w = 12*60*60*1000 # half day in ms
    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
    p = figure(x_axis_type="datetime", tools=TOOLS, height = 500,width=1400, title = "Stock Candlestick")
    p.xaxis.major_label_orientation = pi/4
    p.grid.grid_line_alpha=0.2
    p.segment(df.index, df.High, df.index, df.Low, color="black")
    p.vbar(df.index[inc], w, df.Open[inc], df.Close[inc], fill_color="#98FB98", line_color="black")
    p.vbar(df.index[dec], w, df.Open[dec], df.Close[dec], fill_color="#FF7F7F", line_color="black")

    # Adding EMA lines
    p.line(df.index, signals['Fast_EMA'], legend_label="EMA-Fast", line_width=2, line_color="red")
    p.line(df.index, signals['Slow_EMA'], legend_label="EMA-Slow", line_width=2, line_color="blue")

    # Adding Volume Bars
    p1 = figure(x_axis_type="datetime", height=150, width=1400,x_range=p.x_range, title="Volume", tools=TOOLS)
    p1.vbar(x=df.index, top=df['Volume']/10**7)

    # Adding RSI Signals
    p2 = figure(x_axis_type="datetime", height=100, width=1400,x_range=p.x_range ,title="RSI", tools=TOOLS)
    p2.line(df.index, rsi['RSI'], legend_label="RSI", line_width=2, line_color="blue")
    p2.line(df.index, 30, legend_label="Lower", line_width=2, line_color="green")
    p2.line(df.index, 80, legend_label="Upper", line_width=2, line_color="red")

    # Adding buy/sell signals
    a = signals.loc[signals['positions'] == 1]
    b = signals.loc[signals['positions'] == -1]
    p.scatter(a.index,df['Open'][a.index], marker="triangle",legend_label="Buy", fill_color="blue", size = 9)
    p.scatter(b.index,df['Open'][b.index], marker="triangle",legend_label="Sell", fill_color="red", size = 9)

    # Adding Historical Highs and Lows
    hi = hilo.loc[hilo['positions'] == 1]
    lo = hilo.loc[hilo['positions'] == -1]
    p.scatter(hi.index,df['High'][hi.index], marker="triangle",legend_label="Historical High", fill_color="black", size = 8)
    p.scatter(lo.index,df['Low'][lo.index], marker="triangle",legend_label="Historical Low", fill_color="black", size = 7)
    p.legend.click_policy="hide"

    c = column(result,p,p1,p2)
    pf = c

    output_file("Final_Analysis.html", title="Final_Analysis.py")

    show(pf)  # open a browser

    return


In [27]:
if __name__ == "__main__":

    # Change Values here
    period_start = "2011-09-01"# Using 10 years data 
    period_end = "2021-09-01"
    symbol = read_input()

    # Returns
    Returns = pd.DataFrame(columns=['Fast','Slow','Return', 'BuynHold', 'Better_Strategy'])

    # Dataset
    ohlcv=get_data(symbol,period_start,period_end)
    if ohlcv.empty:
        print("No data found on yFinance")
        exit(0)

    # BnH Returns
    buynhold=bnh(ohlcv)

    # EMA Returns Optimizer
    for i in range(2,21): # buy_fast
        for j in range(11,51): # buy_slow
            if i<j:
                signals = generate_EMA_signals(ohlcv,i,j)
                returns=calculate_pnl(ohlcv,signals)
                Returns.loc[len(Returns.index)] = [i,j,returns,buynhold, 0]
    
    # Finding best combination
    x = Returns.index[Returns.Return.idxmax()]
    if Returns['Return'][x] > Returns['BuynHold'][x]:
        Returns['Better_Strategy'][x] = 1
    max_val = [symbol, Returns['Fast'][x], Returns['Slow'][x], Returns['Return'][x], Returns['BuynHold'][x], Returns['Better_Strategy'][x]]
    print(max_val)

    # For displaying plot:
    signals = generate_EMA_signals(ohlcv,Returns['Fast'][x], Returns['Slow'][x])
    rsi = generate_RSI_signals(ohlcv,30,80)
    hilo = generate_hilo(ohlcv)
    disp(ohlcv, signals, rsi, hilo, max_val)

    # For displaying return per transaction:
    # cal_show_pnl(ohlcv,signals)

Stock Symbol to search: BTC-USD
[*********************100%***********************]  1 of 1 completed
['BTC-USD', 16.0, 50.0, 337270493.1789623, 99940056.46220821, 1.0]
